# OPTaaS - Advanced Options

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

## Connect to the OPTaaS server using your API Key

In [13]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Store additional data in a Task

This can be a JSON, array, string, number or boolean:

In [2]:
from mindfoundry.optaas.client.parameter import FloatParameter

task = client.create_task(
    title='My Task with User-defined Data', 
    parameters=[
        FloatParameter('x', minimum=0, maximum=5),
        FloatParameter('y', minimum=1, maximum=5),
    ],
    user_defined_data={
        'description': 'Lorem ipsum...',
        'tags': ['abc', 'defg']
    }
)

print(task.user_defined_data)

{'description': 'Lorem ipsum...', 'tags': ['abc', 'defg']}


## Warm start

If you've already tried some configurations, you can record the scores upfront, thus giving the optimizer a "warm start":

In [3]:
def scoring_function(x, y):
    return (x * y) - (x / y)

warm_start_values = [
    {'x': 0, 'y': 1},
    {'x': 1, 'y': 2},
    {'x': 0.5, 'y': 1.5},
]
for values in warm_start_values:
    task.add_user_defined_configuration(values, scoring_function(**values))

task.get_results()

[{ 'configuration': '3849a9ca-0ec4-4ef2-8b66-91dce0726876',
   'score': 0.0,
   'user_defined_data': None},
 { 'configuration': 'd38fdd4f-aaad-42dc-8589-7f1f4bf89575',
   'score': 1.5,
   'user_defined_data': None},
 { 'configuration': 'cb145cfc-cb4d-4d25-bc88-f7407a6863fa',
   'score': 0.4166666666666667,
   'user_defined_data': None}]

## Store additional data in a Result

This can be a JSON, array, string, number or boolean:

In [4]:
configuration = task.generate_configurations()[0]
score = scoring_function(**configuration.values)
task.record_result(configuration, score, user_defined_data={'Any data': ['you', 'like']})

task.get_results()[-1:]

[{ 'configuration': '5dfe5518-7e20-4d0b-bf79-711ed780927f',
   'score': 3.9375,
   'user_defined_data': {'Any data': ['you', 'like']}}]

## Report an error

If you encountered an error while calculating the score for a configuration, you can report it:

In [5]:
error_configuration = task.generate_configurations()[0]
next_configuration = task.record_result(error_configuration, 
                                        error='Unexpected error: code 12345')
error_result = task.get_results()[-1]
print(f'Score: {error_result.score}  {error_result.error}')

Score: None  Unexpected error: code 12345


## Get best result and configuration

In [6]:
task.get_best_result_and_configuration()

{ 'configuration': {'type': 'exploitation', 'values': {'x': 1.875, 'y': 2.5}},
  'score': 3.9375,
  'user_defined_data': {'Any data': ['you', 'like']}}

## Get top N results and configurations

In [7]:
task.get_results(limit=3, best_first=True, include_configurations=True)

[{ 'configuration': {'type': 'exploitation', 'values': {'x': 1.875, 'y': 2.5}},
   'score': 3.9375,
   'user_defined_data': {'Any data': ['you', 'like']}},
 { 'configuration': {'type': 'user-defined', 'values': {'x': 1, 'y': 2}},
   'score': 1.5,
   'user_defined_data': None},
 { 'configuration': {'type': 'user-defined', 'values': {'x': 0.5, 'y': 1.5}},
   'score': 0.4166666666666667,
   'user_defined_data': None}]

## Get results and configurations as a Pandas DataFrame

In [9]:
task.get_results(as_dataframe=True)

,config.x,config.y,error,score,variance
0,0.000,1.0,None,0.000000,0.0
1,1.000,2.0,None,1.500000,0.0
2,0.500,1.5,None,0.416667,0.0
3,1.875,2.5,None,3.937500,0.0
4,3.125,1.5,Unexpected error: code 12345,NaN,NaN


## Resume a completed task
Completing a task means that no further configurations can be generated and no further results can be recorded for it.

In [10]:
task.complete()
try:
    task.generate_configurations()
except Exception as err:
    print(err)

Cannot add configurations to a completed task


However, you can resume a completed task if necessary:

In [11]:
task.resume()
task.generate_configurations()

[{'type': 'exploitation', 'values': {'x': 0.9375, 'y': 2.25}}]